# <center> Case - Webmotors <center>

O objetivo deste trabalho é expôr a minha solução para o case da Webmotors apresentado como parte do processo seletivo para Cientista de Dados da GCB Investimentos.

Este notebook está dividido nos seguintes tópicos :

1. **Introdução**


    1.1 Leitura dos Arquivos Disponibilizados
    1.2 Algumas Observações Iniciais
    1.3 Entendendo a Distribuição da Variável Leads
    
2. **Parte 1 - Variação da Quantidade de Leads de acordo com outras Variáveis**


    2.1 cod_marca_veiculo
    2.2 cod_modelo_veiculo
    2.3 uf_cidade
    
3. **Pré-Processamento dos Dados**

    3.1 Tratamento de Valores Ausentes
        3.1.1 vlr_mercado
        3.1.2 flg_ipva_pago
        3.1.3 flg_todas_revisoes_concessionaria
        3.1.4 flg_garantia_fabrica
        3.1.5 flg_adaptado_pcd
        3.1.6 airbag
    3.2 Tratamento de Variáveis Categóricas
        3.2.1 Variáveis que possuem 2 categorias
        3.2.2 Variáveis que possuem mais de 2 Categorias
    3.3 Tratamento de Outliers
        3.3.1 klm_veiculo
        3.3.2 ano_modelo
        3.3.3 vlr_anuncio
        3.3.4 vlr_mercado
    3.4 Feature Scaling
     
        
4. **Desafio Bônus**


    4.1 Seleção das Features com RFECV
    4.2 Validação - Cruzada
    4.3 Gerando Previsões em um Conjunto de Teste
    4.4 Otimização de Hiperparâmetros com GridSearchCV
    4.5 Conclusão
        
5. **Parte 2 - Modelo para determinar se um Anúncio receberá um Lead**


    5.1 Considerações Iniciais
    5.2 Oversampling
    5.3 Medindo a Performance pós Oversampling
    5.4 Feature Selection
    5.5 O que poderia nos levar a conclusões melhores ?

In [1]:
# Algumas importações básicas

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (mean_squared_error , mean_absolute_error , r2_score , accuracy_score , precision_score ,
recall_score , f1_score)
from sklearn.model_selection import cross_val_score , train_test_split , GridSearchCV
from lightgbm import LGBMRegressor , LGBMClassifier
from sklearn.linear_model import LinearRegression , LogisticRegression
from xgboost import XGBRegressor , XGBClassifier
from catboost import CatBoostRegressor , CatBoostClassifier
from sklearn.ensemble import (GradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingClassifier,
RandomForestClassifier, AdaBoostClassifier)
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor , DecisionTreeClassifier
from sklearn.feature_selection import RFECV
from imblearn.over_sampling import RandomOverSampler

# <center> 1. Introdução <center>

## 1.1 Leitura dos Arquivos Disponibilizados

In [2]:
# Lendo o Dicionário de Dados

pd.read_csv("Case 1 - dicionario.csv")

,attribute,description
0,cod_anuncio,código do anúncio
1,cod_cliente,código do anunciante
2,cod_tipo_pessoa,"tipo de anunciante: PF=1, PJ=2"
3,prioridade,"prioridade do anúncio (1=alta, 2-média, 3-baixa)"
4,leads,tota de propostas recebidas
5,views,quantidade de visualizações no anúncio
6,cliques_telefone,quantidade de cliques no telefone anunciado
7,cod_marca_veiculo,código da marca do veículo
8,cod_modelo_veiculo,código do modelo veículo
9,cod_versao_veiculo,código da versão do veículo


In [3]:
# Lendo o arquivo que contém os dados, salvando este como um dataframe e exibindo o dataframe

df = pd.read_csv("Case 1 - dados.csv")

df

,cod_anuncio,cod_cliente,cod_tipo_pessoa,prioridade,leads,views,cliques_telefone*,cod_marca_veiculo,cod_modelo_veiculo,cod_versao_veiculo,...,arquente,bancocouro,arcondic,abs,desembtras,travaeletr,vidroseletr,rodasliga,sensorchuva,sensorestacion
0,2,7941,2,3,1,0,0,34,769,346932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,1770,1,3,1,0,4,2,704,340148,...,S,N,N,S,N,S,S,N,N,N
2,8,709,1,3,4,0,11,30,1071,310713,...,S,N,S,S,S,S,S,S,N,N
3,13,9708,1,2,12,0,14,10,2028,341195,...,S,S,S,N,N,S,S,S,N,N
4,15,1363,1,2,8,0,11,12,805,332063,...,S,S,S,S,S,S,S,S,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48660,195918,3065,2,3,1,34184,0,280,3365,346359,...,S,S,S,S,S,S,S,S,N,S
48661,195924,12535,2,3,1,45048,0,4,1177,343882,...,N,N,N,N,N,N,N,N,N,N
48662,195930,2284,2,3,1,71020,0,5,3128,344252,...,N,N,N,N,N,N,N,N,N,N
48663,195934,2284,2,3,0,82940,0,2,3323,344421,...,S,N,S,N,S,S,S,S,N,N


## 1.2 Algumas Observações Iniciais

In [4]:
# Olhando as 5 primeiras linhas do dataset

df.head()

,cod_anuncio,cod_cliente,cod_tipo_pessoa,prioridade,leads,views,cliques_telefone*,cod_marca_veiculo,cod_modelo_veiculo,cod_versao_veiculo,...,arquente,bancocouro,arcondic,abs,desembtras,travaeletr,vidroseletr,rodasliga,sensorchuva,sensorestacion
0,2,7941,2,3,1,0,0,34,769,346932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,1770,1,3,1,0,4,2,704,340148,...,S,N,N,S,N,S,S,N,N,N
2,8,709,1,3,4,0,11,30,1071,310713,...,S,N,S,S,S,S,S,S,N,N
3,13,9708,1,2,12,0,14,10,2028,341195,...,S,S,S,N,N,S,S,S,N,N
4,15,1363,1,2,8,0,11,12,805,332063,...,S,S,S,S,S,S,S,S,N,N


In [5]:
# Dimensão do conjunto de dados

df.shape

(48665, 41)

Temos 48665 amostras de anúncios e cada um destes possui 41 variáveis descrevendo-o.

In [6]:
# Observando os tipos de dado

df.dtypes

cod_anuncio                            int64
cod_cliente                            int64
cod_tipo_pessoa                        int64
prioridade                             int64
leads                                  int64
views                                  int64
cliques_telefone*                      int64
cod_marca_veiculo                      int64
cod_modelo_veiculo                     int64
cod_versao_veiculo                     int64
ano_modelo                             int64
cep_2dig                               int64
uf_cidade                             object
vlr_anuncio                          float64
qtd_fotos                              int64
km_veiculo                             int64
vlr_mercado                          float64
flg_unico_dono                         int64
flg_licenciado                         int64
flg_ipva_pago                        float64
flg_todas_revisoes_concessionaria    float64
flg_todas_revisoes_agenda_veiculo      int64
flg_garant

In [7]:
# Algumas estatísticas básicas

df.describe()

,cod_anuncio,cod_cliente,cod_tipo_pessoa,prioridade,leads,views,cliques_telefone*,cod_marca_veiculo,cod_modelo_veiculo,cod_versao_veiculo,...,flg_unico_dono,flg_licenciado,flg_ipva_pago,flg_todas_revisoes_concessionaria,flg_todas_revisoes_agenda_veiculo,flg_garantia_fabrica,flg_blindado,flg_aceita_troca,flg_adaptado_pcd,portas
count,48665.000000,48665.000000,48665.000000,48665.000000,48665.000000,48665.000000,48665.000000,48665.000000,48665.000000,48665.000000,...,48665.000000,48665.000000,48353.000000,48353.000000,48665.000000,48353.000000,48665.000000,48665.000000,48353.000000,48664.000000
mean,98293.326518,8804.717374,1.847940,2.601397,4.416521,446.328737,7.467687,19.282791,2063.215658,342632.759149,...,0.207089,0.304099,0.401340,0.151283,0.066742,0.085786,0.020836,0.085523,0.000414,3.727170
std,56501.266859,7954.688001,0.359083,0.767795,11.961256,1105.247844,11.448334,37.630803,1112.006604,13247.921372,...,0.405224,0.460030,0.490175,0.358328,0.249577,0.280050,0.142838,0.279662,0.020334,0.822248
min,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,49530.000000,3630.000000,2.000000,3.000000,1.000000,76.000000,2.000000,4.000000,788.000000,342364.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
50%,98030.000000,6778.000000,2.000000,3.000000,1.000000,196.000000,4.000000,5.000000,2487.000000,344132.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
75%,147292.000000,10504.000000,2.000000,3.000000,4.000000,452.000000,9.000000,26.000000,3202.000000,345633.000000,...,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
max,195939.000000,38733.000000,2.000000,3.000000,364.000000,93608.000000,457.000000,320.000000,3695.000000,347841.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000


In [8]:
# Número de valores ausentes por variável

df.isnull().sum()

cod_anuncio                             0
cod_cliente                             0
cod_tipo_pessoa                         0
prioridade                              0
leads                                   0
views                                   0
cliques_telefone*                       0
cod_marca_veiculo                       0
cod_modelo_veiculo                      0
cod_versao_veiculo                      0
ano_modelo                              0
cep_2dig                                0
uf_cidade                               0
vlr_anuncio                             0
qtd_fotos                               0
km_veiculo                              0
vlr_mercado                          6217
flg_unico_dono                          0
flg_licenciado                          0
flg_ipva_pago                         312
flg_todas_revisoes_concessionaria     312
flg_todas_revisoes_agenda_veiculo       0
flg_garantia_fabrica                  312
flg_blindado                      

<p style='text-align: justify;'>Possuimos valores ausentes em várias variáveis, porém na maioria dessas o número de valores faltantes é bem pequeno quando comparamos com a dimensão do dataset. Repare que possuimos somente 5 variáveis onde o número de valores ausentes é superior a 300 (destaque para a variável vlr_mercado que possui 6217 valores ausentes).

O tratamento dos valores ausentes será realizado mais adiante.

## 1.3 Entendendo a Distribuição da Variável Leads

In [9]:
# Distribuição da Quantidade de leads

df['leads'].value_counts()

1      23394
0      10633
4       6507
8       2870
12      1586
       ...  
252        1
160        1
172        1
156        1
140        1
Name: leads, Length: 64, dtype: int64

In [10]:
# Percentual de anúncios que não recebem nenhum Lead 

(10633/48665)*100

21.84937840336998

In [11]:
## Percentual de anúncios que recebem pelo menos um Lead

100 - 21.84937840336998

78.15062159663002

Repare que aproximadamente **78 % dos anúncios recebem pelo menos 1 Lead**.

In [12]:
# Percentual de anúncios que recebem no máximo 1 lead (recebem 0 ou 1 lead)

((23394 + 10633)/48665)*100

69.92088770163362

Agora veja que aproximadamente 70% recebem no máximo 1 Lead e, portanto, apenas 30 % dos anúncios recebem mais que 1 Lead.

In [13]:
# Número médio de leads por anúncio

df['leads'].mean()

4.4165211137367715

# <center> 2. Parte 1 - Variação da Quantidade de Leads de acordo com outras Variáveis<center>

## 2.1 cod_marca_veiculo

A variável cod_marca_veiculo indica a marca do veículo.

In [14]:
# Calculando quantas marcas existem em nosso dataset

len(np.unique(df['cod_marca_veiculo']))

75

Existem 75 marcas distintas !

In [15]:
# Olhando para a distribuição da variável

df['cod_marca_veiculo'].value_counts()

2      6843
5      6680
4      6454
3      4636
30     3616
       ... 
320       1
176       1
232       1
246       1
219       1
Name: cod_marca_veiculo, Length: 75, dtype: int64

<p style='text-align: justify;'>Apesar de existirem 75 marcas percebemos a cima que a maioria dos anúncios fazem referência a marcas específicas. Por exemplo somente a marca de código 2 está presente em 6843 anúncios.

In [16]:
# Calculando a quantidade de marcas que possuem mais de 1000 anúncios

sum(df['cod_marca_veiculo'].value_counts() > 1000)

12

Somente 12 das 75 marcas possuem mais que 1000 anúncios.

In [17]:
# Percentual de anúncios referente a essas 12 marcas

(sum(df['cod_marca_veiculo'].value_counts()[0:12])/48665)*100

86.88996198499949

Aproximadamente **87% dos anúncios são dessas 12 marcas**. Portanto podemos considerar que os anúncios dessas 12 marcas representam fortemente nosso dataset. Vamos agora avaliar como a quantidade de Leads está distribuída para anúncios dessas 12 marcas.

In [18]:
códigos = list(df['cod_marca_veiculo'].value_counts()[0:12].index)

n_veículos = []

n_leads = []

for código in códigos :
    n_veículos.append(sum(df['cod_marca_veiculo'] == código))
    n_leads.append(df[df['cod_marca_veiculo'] == código]['leads'].sum()) 

média_leads_por_marca = np.array(n_leads)/np.array(n_veículos)

(pd.DataFrame({'Código da Marca' : códigos , "N° Anúncios" : n_veículos , 'Total de Leads' : n_leads , 'Média de Leads' :
              média_leads_por_marca})).sort_values(by = 'Média de Leads' , ascending = False)

,Código da Marca,N° Anúncios,Total de Leads,Média de Leads
6,16,2779,20706,7.450882
11,26,1261,6563,5.204600
5,17,3224,15912,4.935484
7,35,2620,12260,4.679389
9,28,1364,5689,4.170821
1,5,6680,27213,4.073802
10,27,1327,5343,4.026375
3,3,4636,18537,3.998490
0,2,6843,26911,3.932632
8,12,1481,4966,3.353140


<p style='text-align: justify;'>Como podemos ver na tabela a cima o código da marca do veículo não influencia tanto assim o número de leads que o anúncio recebe. A maioria das marcas recebem em média algo em torno de 4 leads. Das 12 marcas 9 recebem em média algo entre 3 e 5 leads . Apenas a marca de código 30 recebe em média menos que 3 leads e apenas as marcas de código 16 e 26 recebem em média mais do que 5 leads. Destaque para a marca de código 16 que recebe em média 7.45 leads (um número expressivo que demonstra que essa marca é bastante procurada).

## 2.2 cod_modelo_veiculo

A variável cod_modelo_veiculo indica o modelo do veículo.

In [19]:
# Calculando quantos modelos de veículo existem em nosso dataset

len(np.unique(df['cod_modelo_veiculo']))

656

Existem 656 modelos distintos de veículo !

In [20]:
# Olhando para a distribuição da variável

df['cod_modelo_veiculo'].value_counts()

2848    1494
740     1458
1222    1222
788     1214
687     1198
        ... 
805        1
2171       1
2655       1
2151       1
3293       1
Name: cod_modelo_veiculo, Length: 656, dtype: int64

In [21]:
(sum(df['cod_modelo_veiculo'].value_counts()[0:31]))/48665

0.5015514229939382

Repare que o código logo a cima nos diz que os 30 modelos que mais aparecem no dataset representam sozinhos aproximadamente 50% dos dados.

Vamos ver abaixo como é a distribuição do número de leads para carros desses 30 modelos :

In [22]:
códigos = list(df['cod_modelo_veiculo'].value_counts()[0:31].index)

n_veículos = []

n_leads = []

for código in códigos :
    n_veículos.append(sum(df['cod_modelo_veiculo'] == código))
    n_leads.append(df[df['cod_modelo_veiculo'] == código]['leads'].sum()) 

média_leads_por_modelo = np.array(n_leads)/np.array(n_veículos)

(pd.DataFrame({'Código do Modelo' : códigos , "N° Anúncios" : n_veículos , 'Total de Leads' : n_leads , 'Média de Leads' :
              média_leads_por_modelo})).sort_values(by = 'Média de Leads' , ascending = False)

,Código do Modelo,N° Anúncios,Total de Leads,Média de Leads
9,690,976,7982,8.178279
12,1194,746,5927,7.945040
30,2862,437,2473,5.659039
24,1143,489,2764,5.652352
23,3511,525,2474,4.712381
6,776,1151,5410,4.700261
22,761,534,2443,4.574906
26,2600,466,2114,4.536481
28,778,457,2061,4.509847
13,703,725,2946,4.063448


<p style='text-align: justify;'>Repare que dos 30 modelos que mais aparecem nos anúncios (representantes de 50% dos dados) 28 recebem algo entre 2 e 6 leads em média. Levando em conta que ao considerarmos o dataset inteiro o número médio de leads por anúncio é ~ 4.42 podemos ver que o modelo não afeta tanto assim o número de leads de um carro. 

## 2.3 uf_cidade

Essa variável descreve a Unidade da Federação e a cidade do anunciante.

Como existem muitas cidades vamos concentrar nosso estudo na distribuição do número de leads por UF.

In [23]:
df['uf_cidade']

0              GO_Itumbiara
1                 SP_Osasco
2             SP_SÃ£o Paulo
3             SP_SÃ£o Paulo
4             SP_SÃ£o Paulo
                ...        
48660               SP_JaÃº
48661    RJ_Duque de Caxias
48662           SP_JundiaÃ­
48663           SP_JundiaÃ­
48664           SP_JundiaÃ­
Name: uf_cidade, Length: 48665, dtype: object

In [24]:
# Dividindo em UF e cidade

uf_cidade_split = df['uf_cidade'].str.split('_')

uf_cidade_split

0              [GO, Itumbiara]
1                 [SP, Osasco]
2             [SP, SÃ£o Paulo]
3             [SP, SÃ£o Paulo]
4             [SP, SÃ£o Paulo]
                 ...          
48660               [SP, JaÃº]
48661    [RJ, Duque de Caxias]
48662           [SP, JundiaÃ­]
48663           [SP, JundiaÃ­]
48664           [SP, JundiaÃ­]
Name: uf_cidade, Length: 48665, dtype: object

In [25]:
# Extraindo a UF da variável uf_cidade

ufs = []

for uf_cidade in uf_cidade_split :
    ufs.append(uf_cidade[0])

In [26]:
# Exibindo o resultado

ufs

['GO',
 'SP',
 'SP',
 'SP',
 'SP',
 'RS',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'MG',
 'SP',
 'SP',
 'DF',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'RJ',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'RJ',
 'SP',
 'SP',
 'SP',
 'SP',
 'MG',
 'GO',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'RJ',
 'SP',
 'SP',
 'PI',
 'SP',
 'SP',
 'MG',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'PA',
 'SP',
 'SP',
 'SP',
 'SP',
 'RJ',
 'SP',
 'SP',
 'PA',
 'SP',
 'SP',
 'PR',
 'SC',
 'RJ',
 'SP',
 'RS',
 'RS',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'RS',
 'MG',
 'PR',
 'SP',
 'DF',
 'PR',
 'RJ',
 'SP',
 'SP',
 'SP',
 'PR',
 'RJ',
 'SP',
 'SP',
 'SP',
 'SP',
 'RN',
 'SP',
 'SP',
 'SP',
 'SP',
 'ES',
 'SP',
 'SC',
 'SP',
 'SP',
 'SC',
 'SP',
 'SP',
 'SP',
 'RJ',
 'SP',
 'SP',
 'RJ',
 'SP',
 'RJ',
 'MG',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',
 'SP',

In [27]:
# Eliminando a feature uf_cidade do df , pois ela não é mais útil

df.drop('uf_cidade' , axis = 1 , inplace = True)

In [28]:
# Criando a feature UF que indica a unidade da Federação do anúncio

df['UF'] = ufs

Vamos abaixo ver como é de fato a distribuição do número de Leads por UF :

In [29]:
ufs = list(np.unique(ufs))

n_veículos = []

n_leads = []

for uf in ufs :
    n_veículos.append(sum(df['UF'] == uf))
    n_leads.append(df[df['UF'] == uf]['leads'].sum()) 

média_leads_por_uf = np.array(n_leads)/np.array(n_veículos)

(pd.DataFrame({'UF' : ufs , "N° Anúncios" : n_veículos , 'Total de Leads' : n_leads , 'Média de Leads' :
              média_leads_por_uf})).sort_values(by = 'Média de Leads' , ascending = False)

,UF,N° Anúncios,Total de Leads,Média de Leads
26,SP,22918,151382,6.605376
19,RJ,3717,15177,4.083132
0,AC,32,114,3.562500
10,MG,3243,9973,3.075239
1,AL,80,242,3.025000
8,GO,931,2390,2.567132
4,BA,632,1614,2.553797
6,DF,1758,4305,2.448805
20,RN,530,1287,2.428302
15,PB,236,549,2.326271


In [30]:
# Percentual de anunciantes em SP

(22918/48665)*100

47.09339360937018

In [31]:
# Percentual de anunciantes na região Sudeste

((22918 + 3717 + 3243 + 472)/48665)*100

62.36514949142094

In [32]:
# Percentual de anunciantes na região Sul

((4440 + 2609 + 3760)/48665)*100

22.21103462447344

In [33]:
# Percentual de anunciantes nas regiões Sul e Sudeste juntas

22.21 + 62.37

84.58

<p style='text-align: justify;'>O que vemos aqui é um comportamento bem diferente entre os estados ! Perceba que quase metade de todos os anúncios são de SP (47%) . Além disso, somente na região Sudeste temos 62% dos anúncios e quando pensamos nas duas regiões mais desenvolvidas do país juntas chegamos a ~ 84% . É importante destacar também que o número médio de leads por anúncio é visivelmente bem mais elevado na região Sudeste que possui 3 dos 4 estados melhores colocados nesse quesito (Acre está a frente de MG provavelmente porque sua amostra é pequena). Na região Sul apesar do alto número de anúncios não temos um número de leads médio tão considerável como na região Sudeste. Por fim vale destacar a liderança com muita margem de SP frente aos outros estados tanto em número de anúncios como em número médio de leads por anúncio.

# <center> 3. Pré-Processamento dos Dados <center>

## 3.1 Tratamento de Valores Ausentes

In [34]:
# Observando novamente as features com valores ausentes

df.isnull().sum()

cod_anuncio                             0
cod_cliente                             0
cod_tipo_pessoa                         0
prioridade                              0
leads                                   0
views                                   0
cliques_telefone*                       0
cod_marca_veiculo                       0
cod_modelo_veiculo                      0
cod_versao_veiculo                      0
ano_modelo                              0
cep_2dig                                0
vlr_anuncio                             0
qtd_fotos                               0
km_veiculo                              0
vlr_mercado                          6217
flg_unico_dono                          0
flg_licenciado                          0
flg_ipva_pago                         312
flg_todas_revisoes_concessionaria     312
flg_todas_revisoes_agenda_veiculo       0
flg_garantia_fabrica                  312
flg_blindado                            0
flg_aceita_troca                  

Podemos reparar abaixo que muitas das features possuem 1 valor ausente somente e **esse valor ausente corresponde a primeira linha do nosso conjunto de dados**.

In [35]:
# Primeira linha do df

df.head(1)

,cod_anuncio,cod_cliente,cod_tipo_pessoa,prioridade,leads,views,cliques_telefone*,cod_marca_veiculo,cod_modelo_veiculo,cod_versao_veiculo,...,bancocouro,arcondic,abs,desembtras,travaeletr,vidroseletr,rodasliga,sensorchuva,sensorestacion,UF
0,2,7941,2,3,1,0,0,34,769,346932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GO


Excluir apenas uma observação não fará falta afinal são mais de 48000 amostras. Melhor do que imputar vários campos em apenas uma linha do dataset.

In [36]:
# Excluindo a primeira linha do df

df.drop(0 , axis = 0 , inplace = True)

In [37]:
# Verificando que deu certo

df.head()

,cod_anuncio,cod_cliente,cod_tipo_pessoa,prioridade,leads,views,cliques_telefone*,cod_marca_veiculo,cod_modelo_veiculo,cod_versao_veiculo,...,bancocouro,arcondic,abs,desembtras,travaeletr,vidroseletr,rodasliga,sensorchuva,sensorestacion,UF
1,6,1770,1,3,1,0,4,2,704,340148,...,N,N,S,N,S,S,N,N,N,SP
2,8,709,1,3,4,0,11,30,1071,310713,...,N,S,S,S,S,S,S,N,N,SP
3,13,9708,1,2,12,0,14,10,2028,341195,...,S,S,N,N,S,S,S,N,N,SP
4,15,1363,1,2,8,0,11,12,805,332063,...,S,S,S,S,S,S,S,N,N,SP
5,18,1880,2,3,0,0,4,2,1042,337626,...,S,S,N,S,S,S,S,N,N,RS


In [38]:
# Agora temos apenas 6 features com valores ausentes

df.isnull().sum()

cod_anuncio                             0
cod_cliente                             0
cod_tipo_pessoa                         0
prioridade                              0
leads                                   0
views                                   0
cliques_telefone*                       0
cod_marca_veiculo                       0
cod_modelo_veiculo                      0
cod_versao_veiculo                      0
ano_modelo                              0
cep_2dig                                0
vlr_anuncio                             0
qtd_fotos                               0
km_veiculo                              0
vlr_mercado                          6216
flg_unico_dono                          0
flg_licenciado                          0
flg_ipva_pago                         312
flg_todas_revisoes_concessionaria     312
flg_todas_revisoes_agenda_veiculo       0
flg_garantia_fabrica                  312
flg_blindado                            0
flg_aceita_troca                  

### 3.1.1 vlr_mercado

Temos 6216 valores ausentes para essa feature. Vamos substituí-los pelos respectivos valores da feature vlr_anuncio, pois o valor que está sendo anunciado é uma boa estimativa para o valor de mercado. Essa é uma opção melhor do que imputar a média da variável vlr_mercado nos campos ausentes.

In [39]:
# Substituindo

df.loc[df['vlr_mercado'].isnull() , 'vlr_mercado'] = df[df['vlr_mercado'].isnull()]['vlr_anuncio']

In [40]:
# Não temos mais nenhum valor ausente nessa coluna !

df['vlr_mercado'].isnull().sum()

0

### 3.1.2 flg_ipva_pago

In [41]:
# Olhando para a distribuição dessa feature

df['flg_ipva_pago'].value_counts()

0.0    28946
1.0    19406
Name: flg_ipva_pago, dtype: int64

Vamos substituir os valores ausentes pela classe que mais ocorre, isto é, por 0 :

In [42]:
# Substituindo !

df.loc[df['flg_ipva_pago'].isnull() , 'flg_ipva_pago'] = 0

In [43]:
# Não temos mais nenhum valor ausente nessa coluna !

df['flg_ipva_pago'].isnull().sum()

0

### 3.1.3 flg_todas_revisoes_concessionaria

In [44]:
# Observando a distribuição dessa feature

df['flg_todas_revisoes_concessionaria'].value_counts()

0.0    41037
1.0     7315
Name: flg_todas_revisoes_concessionaria, dtype: int64

In [45]:
# Substituindo valores ausentes pela classe mais comum, ou seja, por 0

df.loc[df['flg_todas_revisoes_concessionaria'].isnull() , 'flg_todas_revisoes_concessionaria'] = 0

In [46]:
# Não temos mais nenhum valor ausente nessa coluna !

df['flg_todas_revisoes_concessionaria'].isnull().sum()

0

### 3.1.4 flg_garantia_fabrica

In [47]:
# Observando a distribuição dessa feature

df['flg_garantia_fabrica'].value_counts()

0.0    44204
1.0     4148
Name: flg_garantia_fabrica, dtype: int64

In [48]:
# Substituindo pela classe mais comum

df.loc[df['flg_garantia_fabrica'].isnull() , 'flg_garantia_fabrica'] = 0

In [49]:
# Não temos mais nenhum valor ausente nessa coluna !

df['flg_garantia_fabrica'].isnull().sum()

0

### 3.1.5 flg_adaptado_pcd

In [50]:
# Observando a distribuição dessa feature

df['flg_adaptado_pcd'].value_counts()

0.0    48332
1.0       20
Name: flg_adaptado_pcd, dtype: int64

In [51]:
# Substituindo pela classe mais comum

df.loc[df['flg_adaptado_pcd'].isnull() , 'flg_adaptado_pcd'] = 0

In [52]:
# Não temos mais nenhum valor ausente nessa coluna !

df['flg_adaptado_pcd'].isnull().sum()

0

### 3.1.6 airbag

In [53]:
# Observando a distribuição dessa feature

df['airbag'].value_counts()

S    32864
N    15793
Name: airbag, dtype: int64

In [54]:
# Substituindo pela classe mais comum, ou seja, por S

df.loc[df['airbag'].isnull() , 'airbag'] = 'S'

In [55]:
# Não temos mais nenhum valor ausente nessa coluna !

df['airbag'].isnull().sum()

0

Como podemos ver abaixo não temos mais valores faltantes em nosso dataset !

In [56]:
df.isnull().sum()

cod_anuncio                          0
cod_cliente                          0
cod_tipo_pessoa                      0
prioridade                           0
leads                                0
views                                0
cliques_telefone*                    0
cod_marca_veiculo                    0
cod_modelo_veiculo                   0
cod_versao_veiculo                   0
ano_modelo                           0
cep_2dig                             0
vlr_anuncio                          0
qtd_fotos                            0
km_veiculo                           0
vlr_mercado                          0
flg_unico_dono                       0
flg_licenciado                       0
flg_ipva_pago                        0
flg_todas_revisoes_concessionaria    0
flg_todas_revisoes_agenda_veiculo    0
flg_garantia_fabrica                 0
flg_blindado                         0
flg_aceita_troca                     0
flg_adaptado_pcd                     0
combustivel              

## 3.2 Tratamento de Variáveis Categóricas

In [57]:
# Obtendo quem são as features categóricas

features_categoricas = []
for feature , tipo in zip(df.dtypes.index , df.dtypes) :
    if tipo == 'object' :
        features_categoricas.append(feature)

In [58]:
# Observando nossas features categóricas

features_categoricas

['combustivel',
 'cambio',
 'alarme',
 'airbag',
 'arquente',
 'bancocouro',
 'arcondic',
 'abs',
 'desembtras',
 'travaeletr',
 'vidroseletr',
 'rodasliga',
 'sensorchuva',
 'sensorestacion',
 'UF']

### 3.2.1 Variáveis que possuem 2 categorias

Vamos selecionar abaixo as variáveis que possuem 2 categorias apenas. Todas elas assumem apenas o valor "S" ou "N" indicando se possuem ou não determinado atributo.

In [59]:
features_2_categorias = ['alarme', 'airbag', 'arquente', 'bancocouro', 'arcondic', 'abs', 'desembtras', 'travaeletr',
 'vidroseletr', 'rodasliga', 'sensorchuva', 'sensorestacion']

Nossa estratégia será substituir em cada feature S por 1 e N por 0 . Assim estaremos transformando uma feature categórica em numérica e não teremos problemas ao rodar nossos modelos.

In [60]:
# Aplicando a estratégia citada a cima

for feature in features_2_categorias :
    df.loc[df[feature] == 'S' , feature] = 1
    df.loc[df[feature] == 'N' , feature] = 0

In [61]:
# Vendo como ficou

df[features_2_categorias]

,alarme,airbag,arquente,bancocouro,arcondic,abs,desembtras,travaeletr,vidroseletr,rodasliga,sensorchuva,sensorestacion
1,1,1,1,0,0,1,0,1,1,0,0,0
2,1,1,1,0,1,1,1,1,1,1,0,0
3,0,1,1,1,1,0,0,1,1,1,0,0
4,1,0,1,1,1,1,1,1,1,1,0,0
5,0,1,0,1,1,0,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
48660,1,1,1,1,1,1,1,1,1,1,0,1
48661,0,0,0,0,0,0,0,0,0,0,0,0
48662,0,0,0,0,0,0,0,0,0,0,0,0
48663,1,0,1,0,1,0,1,1,1,1,0,0


Perfeito !

### 3.2.2 Variáveis que possuem mais de 2 categorias

As features que possuem mais de 2 categorias são "combustivel" , "cambio" e "UF" .

### cambio

In [62]:
# Observando as categorias da variável câmbio

df['cambio'].value_counts()

manual                   27011
automatico               20866
CVT                        507
automatizado               134
semi-automatico            124
automatico sequencial       11
automatizado DCT            11
Name: cambio, dtype: int64

In [63]:
# Adicionando as variáveis que indicam a presença de cada tipo de cambio ao df

df = pd.concat([df, pd.get_dummies(df['cambio'])] , axis = 1)

In [64]:
# O número de colunas aumente em 7, pois são 7 tipos diferentes de câmbio

df.shape

(48664, 48)

In [65]:
# Observando a estrutura

df[['cambio' , 'manual' , 'automatico' , 'CVT' , 'automatizado' , 'semi-automatico' , 'automatico sequencial' ,
    'automatizado DCT']]

,cambio,manual,automatico,CVT,automatizado,semi-automatico,automatico sequencial,automatizado DCT
1,manual,1,0,0,0,0,0,0
2,manual,1,0,0,0,0,0,0
3,automatico,0,1,0,0,0,0,0
4,automatico,0,1,0,0,0,0,0
5,manual,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
48660,manual,1,0,0,0,0,0,0
48661,manual,1,0,0,0,0,0,0
48662,automatico,0,1,0,0,0,0,0
48663,automatico,0,1,0,0,0,0,0


In [66]:
# Excluindo a variável cambio, pois não precisamos mais dela

df.drop('cambio' , axis = 1 , inplace = True)

### combustível

In [67]:
# Observando as categorias da variável combustível

df['combustivel'].value_counts()

gasolina e alcool                          35195
gasolina                                   10325
diesel                                      2683
gasolina, alcool e gas natural               204
alcool                                       118
gasolina e gas natural                        55
gas natural                                   29
gasolina e eletrico                           29
alcool e gas natural                          18
gasolina, alcool, gas natural e benzina        8
Name: combustivel, dtype: int64

In [68]:
# Adicionando as variáveis que indicam a presença de cada tipo de combustível ao df

df = pd.concat([df, pd.get_dummies(df['combustivel'])] , axis = 1)

In [69]:
# Excluindo a variável combustível, pois não precisamos mais dela

df.drop('combustivel' , axis = 1 , inplace = True)

### UF

In [70]:
# Observando as categorias da variável UF

df['UF'].value_counts()

SP     22918
PR      4440
SC      3760
RJ      3717
MG      3243
RS      2609
DF      1758
GO       930
PE       771
BA       632
MS       553
RN       530
CE       524
ES       472
MT       409
PB       236
MA       232
PA       181
RO       164
AM       158
PI       146
SE       119
AL        80
AC        32
AP        18
RR        15
TO        13
N/A        4
Name: UF, dtype: int64

In [71]:
# Adicionando as variáveis que indicam a presença de cada UF 

df = pd.concat([df, pd.get_dummies(df['UF'])] , axis = 1)

In [72]:
# Excluindo a variável UF, pois não precisamos mais dela

df.drop('UF' , axis = 1 , inplace = True)

## 3.3 Tratamento de Outliers

### 3.3.1 klm_veiculo

<p style='text-align: justify;'>A quilometragem máxima de um veículo tende a ser um número próximo a 400000. Claro que sempre existirão alguns carros que rodarão mais do que 400000 quilômetros (em geral carros mais velho).

In [73]:
# Analisando quantos carros possuem quilometragem superior a 400000 km

(df['km_veiculo'] > 400000).sum()

373

373 veículos é um número muito elevado, deve estar acontecendo alguma coisa com os dados.

Analisando os dados parece que muitos veículos acusam ter 999999 km rodados. 

In [74]:
# Quantos carros possuem 999999 km rodados ?

(df['km_veiculo'] == 999999).sum()

242

In [75]:
# Quantos carros possuem 999999 ou mais km rodados ?

(df['km_veiculo'] >= 999999).sum()

312

312 carros com número de km rodados maior ou igual a 999999 km rodados não parece fazer sentido. Vamos substituir essas quilometragens pela média de km rodados pelos carros dos outros anúncios.

In [76]:
média_de_km_rodados = df[df['km_veiculo'] < 999999]['km_veiculo'].mean()

In [77]:
média_de_km_rodados 

63236.737921906024

63236 parece ser um valor bem mais justo para a quilometragem dos carros.

In [78]:
# Substituindo

df.loc[df['km_veiculo'] >= 999999 , 'km_veiculo'] = média_de_km_rodados

### 3.3.2 ano_modelo

Repare que em duas observações temos valores negativos para o ano do modelo :

In [79]:
sum(df['ano_modelo'] < 0)

2

Nessas duas observações o ano do modelo recebe o valor -1.

In [80]:
df[df['ano_modelo'] < 0]['ano_modelo']

595     -1
35714   -1
Name: ano_modelo, dtype: int64

Vamos substituir -1 pela média dos anos dos outros veículos :

In [81]:
# Calculando a média dos anos

média_ano_modelo = df[df['ano_modelo'] != -1]['ano_modelo'].mean()

In [82]:
média_ano_modelo

2013.1933336073323

In [83]:
df.loc[df['ano_modelo'] == -1 , 'ano_modelo'] = 2013

### 3.3.3 vlr_anuncio

Existem alguns valores no nosso conjunto de dados que estão muito a cima do preço que é considerado condizente com o preço de um automóvel.

Vamos observar os 35 valores mais elevados :

In [84]:
df['vlr_anuncio'].sort_values(ascending = False)[0:35]

28985    405000000.0
17564    218500000.0
20971    195000000.0
15054    158000000.0
45956    122000000.0
35988    122000000.0
30300    113900000.0
36590    108000000.0
16924     90899000.0
47079     87000000.0
22959     82500000.0
41307     81500000.0
46372     81000000.0
45060     79000000.0
44185     74900000.0
47115     54000000.0
26527     53000000.0
16931     50900000.0
18623     49900000.0
22250     46000000.0
19207     45500000.0
36939     43000000.0
4594      42990000.0
33525     33000000.0
39756     29200000.0
34541     28000000.0
21986     27000000.0
22120     25000000.0
45731     18000000.0
940        8888888.0
29754      7100000.0
47085      3299000.0
35977      2860000.0
44616      2149000.0
41897      2149000.0
Name: vlr_anuncio, dtype: float64

Como podemos ver temos 31 carros com valor de anúncio superior a 7 milhões. Inclusive temos valores fora da realidade como 405 milhões.

Iremos substituir os valores de anúncios superiores a 7 milhões pelos seus respectivos valores de mercado.

In [85]:
df.loc[df['vlr_anuncio'] > 7000000 , 'vlr_anuncio'] = df[df['vlr_anuncio'] > 7000000]['vlr_mercado']

In [86]:
# Observando novamente os 35 anúncios mais caros

df['vlr_anuncio'].sort_values(ascending = False)[0:35]

28985    405000000.0
17564    218500000.0
20971    195000000.0
15054    158000000.0
35988    122000000.0
16924     90899000.0
940        8888888.0
47085      3299000.0
35977      2860000.0
41897      2149000.0
44616      2149000.0
45515      1990000.0
34163      1990000.0
42544      1799000.0
41896      1590000.0
47943      1300000.0
2144       1290000.0
39844      1275000.0
43176      1250000.0
99         1235000.0
725        1230000.0
43996      1220000.0
11582      1200000.0
17768      1166900.0
16957      1150000.0
6305       1074800.0
32487      1050000.0
42996      1050000.0
42537      1000000.0
34549       990000.0
2556        980000.0
5321        970000.0
28070       950000.0
13779       940000.0
37850       850000.0
Name: vlr_anuncio, dtype: float64

Ainda temos alguns valores surreais porque nesses casos o valor do anúncio é igual ao valor de mercado (Lembre-se que substituimos os valores ausentes na coluna "vlr_mercado" pelos respectivs valores de anúncio).

Vamos imputar a média dos valores de anúncio nesses valores surreais :

In [87]:
df.loc[df['vlr_anuncio'] > 7000000 , 'vlr_anuncio'] = df[df['vlr_anuncio'] < 7000000]['vlr_anuncio'].mean()

### 3.3.4 vlr_mercado

In [88]:
# Maiores valores de mercado

df['vlr_mercado'].sort_values(ascending = False)[0:10]

28985    405000000.0
17564    218500000.0
20971    195000000.0
15054    158000000.0
35988    122000000.0
16924     90899000.0
940        8888888.0
47085      3299000.0
35977      2860000.0
41897      2149000.0
Name: vlr_mercado, dtype: float64

In [89]:
# Substituindo os maiores valores de mercado pela média dos valores de mercado

df.loc[df['vlr_mercado'] > 7000000 , 'vlr_mercado'] = df[df['vlr_mercado'] < 7000000]['vlr_mercado'].mean()

In [90]:
# Exibindo novamente os maiores valores de mercado

df['vlr_mercado'].sort_values(ascending = False)[0:10]

47085    3299000.0
35977    2860000.0
41897    2149000.0
44616    2149000.0
34163    1990000.0
42544    1799000.0
41896    1590000.0
47943    1300000.0
2144     1290000.0
39844    1275000.0
Name: vlr_mercado, dtype: float64

## 3.4 Feature Scaling

In [91]:
# Instanciando

scaler = StandardScaler()

In [92]:
# Selecionando as features que vamos aplicar o dimensionamento

features_scaler = ['vlr_mercado' , 'km_veiculo' , 'qtd_fotos' , 'vlr_anuncio' , 'cep_2dig' , 'ano_modelo', 
                  'cod_versao_veiculo' , 'cod_modelo_veiculo' , 'cod_marca_veiculo' , 'cliques_telefone*' ,
                   'views' , 'cod_cliente' , 'cod_anuncio']

In [93]:
# Aplicando o dimensionamento padrão em cada uma das features a cima

for feature in features_scaler :
    df[feature] = scaler.fit_transform(np.array(df[feature]).reshape(-1,1))

In [94]:
# Observando como ficaram os valores nas columas em que aplicamos o dimensionamento de variáveis

df[features_scaler]

,vlr_mercado,km_veiculo,qtd_fotos,vlr_anuncio,cep_2dig,ano_modelo,cod_versao_veiculo,cod_modelo_veiculo,cod_marca_veiculo,cliques_telefone*,views,cod_cliente,cod_anuncio
1,-0.801908,12.037971,-3.053281,-0.785747,-0.952290,-3.601467,-0.187552,-1.222350,-0.459265,-0.302914,-0.403836,-0.884351,-1.739650
2,-0.295083,-0.567467,-1.210059,-0.286341,-1.071897,-2.344653,-2.409412,-0.892311,0.284807,0.308532,-0.403836,-1.017731,-1.739614
3,-0.208674,0.802366,0.633162,-0.201196,-1.012093,-3.810935,-0.108521,-0.031693,-0.246673,0.570580,-0.403836,0.113551,-1.739526
4,-0.441315,0.153761,-0.288448,-0.430432,-0.982191,-3.810935,-0.797837,-1.131522,-0.193525,0.308532,-0.403836,-0.935516,-1.739490
5,-0.324994,-1.214919,-1.210059,-0.315814,1.559465,-1.716247,-0.377922,-0.918391,-0.459265,-0.302914,-0.403836,-0.870523,-1.739437
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48660,-0.142016,-0.542491,0.633162,-0.135700,-0.623369,0.587911,0.281277,1.170655,6.928311,-0.652311,30.525026,-0.721554,1.727850
48661,-0.223873,0.537660,-2.592476,-0.217570,-0.384155,-0.040496,0.094304,-0.796987,-0.406117,-0.652311,40.354512,0.468939,1.727956
48662,0.417738,1.090549,-3.053281,0.615864,-0.742977,-0.249965,0.122233,0.957524,-0.379543,-0.652311,63.853353,-0.819735,1.728062
48663,-0.178696,-0.446430,-3.053281,-0.040731,-0.742977,0.168973,0.134990,1.132885,-0.459265,-0.652311,74.638282,-0.819735,1.728133


# <center> 4. Desafio Bônus <center>

## 4.1 Seleção das Features com RFECV

In [95]:
# Criando a lista de todas features que temos a disposição para utilizarmos no treinamento de nossos modelos

features = ['cod_anuncio', 'cod_cliente' , 'cod_tipo_pessoa', 'prioridade',
       'views', 'cliques_telefone*', 'cod_marca_veiculo', 'cod_modelo_veiculo',
       'cod_versao_veiculo', 'ano_modelo', 'cep_2dig', 'vlr_anuncio',
       'qtd_fotos', 'km_veiculo', 'vlr_mercado', 'flg_unico_dono',
       'flg_licenciado', 'flg_ipva_pago', 'flg_todas_revisoes_concessionaria',
       'flg_todas_revisoes_agenda_veiculo', 'flg_garantia_fabrica',
       'flg_blindado', 'flg_aceita_troca', 'flg_adaptado_pcd', 'portas',
       'alarme', 'airbag', 'arquente', 'bancocouro', 'arcondic', 'abs',
       'desembtras', 'travaeletr', 'vidroseletr', 'rodasliga', 'sensorchuva',
       'sensorestacion', 'CVT', 'automatico', 'automatico sequencial',
       'automatizado', 'automatizado DCT', 'manual', 'semi-automatico',
       'alcool', 'alcool e gas natural', 'diesel', 'gas natural', 'gasolina',
       'gasolina e alcool', 'gasolina e eletrico', 'gasolina e gas natural',
       'gasolina, alcool e gas natural',
       'gasolina, alcool, gas natural e benzina' , 'AC', 'AL', 'AM', 'AP',
       'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'N/A', 'PA', 'PB',
       'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO']

In [96]:
# Criando a matriz de features e a variável alvo (leads)

X = np.array(df[features])

y = np.array(df['leads'])

Agora surge a grande pergunta : Quais dessas features devemos selecionar para treinar nossos modelos ?

<p style='text-align: justify;'>Utilizar todas as features disponíveis nem sempre é uma boa idéia. Devemos nos lembrar que algumas features podem estar correlacionadas com outras, ou seja, uma feature pode "carregar consigo" boa parte da informação presente em outra e isso geralmente piora o desempenho dos modelos. Além disso, um número elevado de variávies pode acabar por gerar ruído e piorar a performance dos modelos. Devemos nos lembrar também que se tivermos muitas features o modelo pode acabar "decorando" os exemplos de treino e se ajustar excessivamente ao conjunto de treino (overfitting).

Existem várias maneiras de selecionar as features adequadas :

 - Remover features com baixa variância

- Selecionar com base em testes estatísticos

- Eliminação recursiva de features (RFE)

Escolheremos a última opção para selecionar nossas features. Na verdade utilizaremos um método baseado nessa opção, porém um pouco mais robusto : Recursive Feature Elimination with Cross-Validation (RFECV).

<p style='text-align: justify;'>O objetivo da eliminação recursiva de features (RFE) é selecionar as features considerando recursivamente conjuntos cada vez menores de variáveis. Primeiro, o estimador é treinado no conjunto inicial de features e a importância de cada variável é obtida através de qualquer atributo específico (como coef_, feature_importances_) ou callable. Em seguida, as features menos importantes são removidos do conjunto atual de features. Esse procedimento é repetido recursivamente no conjunto podado até que o número desejado de features a serem selecionadas seja finalmente alcançado. O algoritmo RFECV ainda se utiliza da validação cruzada para determinar o número ótimo de features.

Para rodar o algoritmo RFECV utilizaremos o modelo LightGBM , pois além de ser um modelo que possui o atributo feature_importances ele costuma ter um tempo de treinamento inferior aos demais modelos e como o RFECV é custoso computacionalmente esse modelo se revela uma ótima opção.

<p style='text-align: justify;'>Além disso utilizaremos como métrica a negação do erro absoluto médio. Por padrão do Scikit-Learn o melhor score é sempre o maior, porém sabemos que quanto menor o erro absoluto médio melhor o modelo e, portanto, buscaremos o maior valor da negação do erro absoluto médio, pois quanto maior a negação menor o erro absoluto médio.

In [97]:
# Criando o array com todas as features disponíveis

X = np.array(df[features])

In [98]:
# Instanciando o algoritmo RFECV

selector = RFECV(estimator = LGBMRegressor() , step = 1 , cv = 5 , scoring = 'neg_mean_absolute_error')

In [99]:
# Essa célula demora pra rodar, pois o algoritmo RFECV treina muitos modelos

selector.fit(X, y)

RFECV(cv=5, estimator=LGBMRegressor(), scoring='neg_mean_absolute_error')

In [100]:
# Número de features selecionadas

selector.n_features_

1

Que interessante ! **O RFECV nos diz que o melhor modelo possível é treinado em apenas 1 feature** ! 

Treinar o modelo LGBMRegressor com 1 feature nos traz melhor score do que se treinássemos em todo o conjunto de variáveis !

In [101]:
# Obtendo a máscara de features selecionadas

mask = selector.support_

mask

array([False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [102]:
# Descobrindo qual foi a Feature indicada pelo RFECV

(np.array(features))[mask]

array(['cliques_telefone*'], dtype='<U39')

A melhor feature para rodar nosso modelo é **"cliques_telefone*"** .

Faz sentido ! Realmente o número de cliques no telefone do anunciante é algo que indica que o cliente parece estar perto de fazer uma proposta. Afinal por que ligar se não tiver fortemente interessado no anúncio ?

## 4.2 Validação - Cruzada

Vamos agora obter o score médio de validação cruzada para diferentes modelos treinados somente na feature "cliques_telefone*" . A validação cruzada nos permitirá ver como os modelos estão desempenhando em dados não vistos durante o treinamento e, com isso, poderemos escolher o modelo que obtém as melhores métricas.

In [103]:
# Nova matriz de features 

X = np.array(df['cliques_telefone*']).reshape(-1,1)

In [104]:
# Modelos

modelos = [LGBMRegressor() , LinearRegression() , KNeighborsRegressor() , DecisionTreeRegressor(),
          RandomForestRegressor() , AdaBoostRegressor(), GradientBoostingRegressor(), 
          XGBRegressor(use_label_encoder = False),
          CatBoostRegressor()]

In [105]:
# Nome dos Modelos

nome_modelos = ['LGBM' , 'Linear Regression' , 'KNN' , 'Decision Tree' , 'Random Forest' , 'ADA Boost',
               'Gradient Boosting' , 'XGB' , 'CatBoost']

In [106]:
# Demora um pouco pra rodar

scores_MAE = []
scores_MSE = []
scores_R2 = []

for model in modelos :
    scores_MAE.append(-np.mean(cross_val_score(model, X, y , cv = 5, scoring = 'neg_mean_absolute_error')))
    scores_MSE.append(-np.mean(cross_val_score(model, X, y , cv = 5, scoring = 'neg_mean_squared_error')))
    scores_R2.append(np.mean(cross_val_score(model, X, y , cv = 5, scoring = 'r2')))

Learning rate set to 0.073021
0:	learn: 10.9837389	total: 204ms	remaining: 3m 23s
1:	learn: 10.5414390	total: 223ms	remaining: 1m 51s
2:	learn: 10.1396668	total: 243ms	remaining: 1m 20s
3:	learn: 9.7708279	total: 264ms	remaining: 1m 5s
4:	learn: 9.4339036	total: 283ms	remaining: 56.4s
5:	learn: 9.1321228	total: 303ms	remaining: 50.2s
6:	learn: 8.8544678	total: 332ms	remaining: 47.1s
7:	learn: 8.6129707	total: 352ms	remaining: 43.6s
8:	learn: 8.3945367	total: 368ms	remaining: 40.6s
9:	learn: 8.1964045	total: 385ms	remaining: 38.1s
10:	learn: 8.0243479	total: 407ms	remaining: 36.6s
11:	learn: 7.8703881	total: 428ms	remaining: 35.2s
12:	learn: 7.7293153	total: 447ms	remaining: 33.9s
13:	learn: 7.6049112	total: 467ms	remaining: 32.9s
14:	learn: 7.4924265	total: 489ms	remaining: 32.1s
15:	learn: 7.3925322	total: 515ms	remaining: 31.7s
16:	learn: 7.3041543	total: 532ms	remaining: 30.8s
17:	learn: 7.2250179	total: 552ms	remaining: 30.1s
18:	learn: 7.1555551	total: 569ms	remaining: 29.4s
19:	l

169:	learn: 6.3696087	total: 3.39s	remaining: 16.6s
170:	learn: 6.3691481	total: 3.41s	remaining: 16.5s
171:	learn: 6.3675681	total: 3.43s	remaining: 16.5s
172:	learn: 6.3671238	total: 3.44s	remaining: 16.5s
173:	learn: 6.3666955	total: 3.46s	remaining: 16.4s
174:	learn: 6.3651754	total: 3.48s	remaining: 16.4s
175:	learn: 6.3647621	total: 3.5s	remaining: 16.4s
176:	learn: 6.3643638	total: 3.51s	remaining: 16.3s
177:	learn: 6.3633627	total: 3.53s	remaining: 16.3s
178:	learn: 6.3619393	total: 3.54s	remaining: 16.3s
179:	learn: 6.3605664	total: 3.56s	remaining: 16.2s
180:	learn: 6.3579577	total: 3.59s	remaining: 16.2s
181:	learn: 6.3575735	total: 3.6s	remaining: 16.2s
182:	learn: 6.3562887	total: 3.62s	remaining: 16.2s
183:	learn: 6.3538803	total: 3.64s	remaining: 16.1s
184:	learn: 6.3516250	total: 3.66s	remaining: 16.1s
185:	learn: 6.3504466	total: 3.67s	remaining: 16.1s
186:	learn: 6.3493116	total: 3.69s	remaining: 16s
187:	learn: 6.3472580	total: 3.7s	remaining: 16s
188:	learn: 6.34630

336:	learn: 6.2703931	total: 6.34s	remaining: 12.5s
337:	learn: 6.2703194	total: 6.35s	remaining: 12.4s
338:	learn: 6.2697187	total: 6.37s	remaining: 12.4s
339:	learn: 6.2694414	total: 6.39s	remaining: 12.4s
340:	learn: 6.2693704	total: 6.41s	remaining: 12.4s
341:	learn: 6.2693019	total: 6.42s	remaining: 12.4s
342:	learn: 6.2688788	total: 6.44s	remaining: 12.3s
343:	learn: 6.2686161	total: 6.46s	remaining: 12.3s
344:	learn: 6.2679425	total: 6.47s	remaining: 12.3s
345:	learn: 6.2676371	total: 6.49s	remaining: 12.3s
346:	learn: 6.2668453	total: 6.51s	remaining: 12.3s
347:	learn: 6.2665535	total: 6.53s	remaining: 12.2s
348:	learn: 6.2661216	total: 6.55s	remaining: 12.2s
349:	learn: 6.2652687	total: 6.57s	remaining: 12.2s
350:	learn: 6.2646543	total: 6.58s	remaining: 12.2s
351:	learn: 6.2645883	total: 6.6s	remaining: 12.2s
352:	learn: 6.2642650	total: 6.62s	remaining: 12.1s
353:	learn: 6.2641207	total: 6.63s	remaining: 12.1s
354:	learn: 6.2637393	total: 6.65s	remaining: 12.1s
355:	learn: 6

498:	learn: 6.2336047	total: 9.12s	remaining: 9.15s
499:	learn: 6.2331572	total: 9.14s	remaining: 9.14s
500:	learn: 6.2330807	total: 9.15s	remaining: 9.12s
501:	learn: 6.2328276	total: 9.17s	remaining: 9.1s
502:	learn: 6.2324832	total: 9.19s	remaining: 9.08s
503:	learn: 6.2323250	total: 9.21s	remaining: 9.07s
504:	learn: 6.2322595	total: 9.23s	remaining: 9.05s
505:	learn: 6.2321778	total: 9.24s	remaining: 9.03s
506:	learn: 6.2320630	total: 9.26s	remaining: 9.01s
507:	learn: 6.2318361	total: 9.28s	remaining: 8.99s
508:	learn: 6.2317311	total: 9.29s	remaining: 8.96s
509:	learn: 6.2316300	total: 9.31s	remaining: 8.95s
510:	learn: 6.2315574	total: 9.33s	remaining: 8.93s
511:	learn: 6.2314689	total: 9.35s	remaining: 8.91s
512:	learn: 6.2311777	total: 9.37s	remaining: 8.89s
513:	learn: 6.2309624	total: 9.38s	remaining: 8.87s
514:	learn: 6.2308580	total: 9.4s	remaining: 8.85s
515:	learn: 6.2304269	total: 9.42s	remaining: 8.83s
516:	learn: 6.2300415	total: 9.43s	remaining: 8.81s
517:	learn: 6.

667:	learn: 6.2129206	total: 12s	remaining: 5.97s
668:	learn: 6.2128898	total: 12s	remaining: 5.95s
669:	learn: 6.2128314	total: 12s	remaining: 5.93s
670:	learn: 6.2127734	total: 12.1s	remaining: 5.92s
671:	learn: 6.2125983	total: 12.1s	remaining: 5.9s
672:	learn: 6.2125877	total: 12.1s	remaining: 5.88s
673:	learn: 6.2125322	total: 12.1s	remaining: 5.86s
674:	learn: 6.2124737	total: 12.1s	remaining: 5.84s
675:	learn: 6.2124385	total: 12.2s	remaining: 5.83s
676:	learn: 6.2124282	total: 12.2s	remaining: 5.81s
677:	learn: 6.2122671	total: 12.2s	remaining: 5.79s
678:	learn: 6.2121585	total: 12.2s	remaining: 5.77s
679:	learn: 6.2121411	total: 12.2s	remaining: 5.75s
680:	learn: 6.2121106	total: 12.2s	remaining: 5.73s
681:	learn: 6.2120547	total: 12.3s	remaining: 5.72s
682:	learn: 6.2118826	total: 12.3s	remaining: 5.7s
683:	learn: 6.2118536	total: 12.3s	remaining: 5.68s
684:	learn: 6.2117275	total: 12.3s	remaining: 5.66s
685:	learn: 6.2115590	total: 12.3s	remaining: 5.64s
686:	learn: 6.211549

...

318:	learn: 4.9090698	total: 5.53s	remaining: 11.8s
319:	learn: 4.9082584	total: 5.55s	remaining: 11.8s
320:	learn: 4.9076592	total: 5.57s	remaining: 11.8s
321:	learn: 4.9069322	total: 5.58s	remaining: 11.8s
322:	learn: 4.9063358	total: 5.6s	remaining: 11.7s
323:	learn: 4.9056435	total: 5.62s	remaining: 11.7s
324:	learn: 4.9051624	total: 5.64s	remaining: 11.7s
325:	learn: 4.9044624	total: 5.66s	remaining: 11.7s
326:	learn: 4.9038151	total: 5.67s	remaining: 11.7s
327:	learn: 4.9031641	total: 5.69s	remaining: 11.7s
328:	learn: 4.9025081	total: 5.71s	remaining: 11.6s
329:	learn: 4.9021796	total: 5.72s	remaining: 11.6s
330:	learn: 4.9020389	total: 5.74s	remaining: 11.6s
331:	learn: 4.9019877	total: 5.76s	remaining: 11.6s
332:	learn: 4.9017958	total: 5.78s	remaining: 11.6s
333:	learn: 4.9011678	total: 5.8s	remaining: 11.6s
334:	learn: 4.9008517	total: 5.82s	remaining: 11.5s
335:	learn: 4.9002505	total: 5.83s	remaining: 11.5s
336:	learn: 4.9000660	total: 5.85s	remaining: 11.5s
337:	learn: 4.

486:	learn: 4.8526465	total: 8.41s	remaining: 8.86s
487:	learn: 4.8523704	total: 8.43s	remaining: 8.84s
488:	learn: 4.8521940	total: 8.45s	remaining: 8.83s
489:	learn: 4.8518688	total: 8.46s	remaining: 8.81s
490:	learn: 4.8516135	total: 8.48s	remaining: 8.79s
491:	learn: 4.8513874	total: 8.5s	remaining: 8.78s
492:	learn: 4.8509709	total: 8.52s	remaining: 8.76s
493:	learn: 4.8508016	total: 8.53s	remaining: 8.74s
494:	learn: 4.8504151	total: 8.55s	remaining: 8.72s
495:	learn: 4.8503910	total: 8.57s	remaining: 8.71s
496:	learn: 4.8502787	total: 8.59s	remaining: 8.69s
497:	learn: 4.8502158	total: 8.61s	remaining: 8.68s
498:	learn: 4.8498931	total: 8.63s	remaining: 8.66s
499:	learn: 4.8498704	total: 8.64s	remaining: 8.64s
500:	learn: 4.8496264	total: 8.66s	remaining: 8.63s
501:	learn: 4.8492229	total: 8.68s	remaining: 8.61s
502:	learn: 4.8490604	total: 8.7s	remaining: 8.59s
503:	learn: 4.8487538	total: 8.71s	remaining: 8.57s
504:	learn: 4.8482967	total: 8.73s	remaining: 8.55s
505:	learn: 4.

652:	learn: 4.8227317	total: 11.3s	remaining: 5.99s
653:	learn: 4.8224218	total: 11.3s	remaining: 5.97s
654:	learn: 4.8221343	total: 11.3s	remaining: 5.96s
655:	learn: 4.8219170	total: 11.3s	remaining: 5.94s
656:	learn: 4.8216841	total: 11.3s	remaining: 5.92s
657:	learn: 4.8216237	total: 11.4s	remaining: 5.91s
658:	learn: 4.8214644	total: 11.4s	remaining: 5.89s
659:	learn: 4.8213762	total: 11.4s	remaining: 5.87s
660:	learn: 4.8211709	total: 11.4s	remaining: 5.85s
661:	learn: 4.8210372	total: 11.4s	remaining: 5.83s
662:	learn: 4.8209767	total: 11.4s	remaining: 5.82s
663:	learn: 4.8209651	total: 11.5s	remaining: 5.8s
664:	learn: 4.8208026	total: 11.5s	remaining: 5.78s
665:	learn: 4.8206322	total: 11.5s	remaining: 5.76s
666:	learn: 4.8204534	total: 11.5s	remaining: 5.75s
667:	learn: 4.8201965	total: 11.5s	remaining: 5.73s
668:	learn: 4.8200558	total: 11.6s	remaining: 5.72s
669:	learn: 4.8200446	total: 11.6s	remaining: 5.7s
670:	learn: 4.8199826	total: 11.6s	remaining: 5.68s
671:	learn: 4.

815:	learn: 4.8041563	total: 14.1s	remaining: 3.19s
816:	learn: 4.8040762	total: 14.1s	remaining: 3.17s
817:	learn: 4.8040191	total: 14.2s	remaining: 3.15s
818:	learn: 4.8039791	total: 14.2s	remaining: 3.13s
819:	learn: 4.8039586	total: 14.2s	remaining: 3.12s
820:	learn: 4.8038825	total: 14.2s	remaining: 3.1s
821:	learn: 4.8038280	total: 14.2s	remaining: 3.08s
822:	learn: 4.8037761	total: 14.3s	remaining: 3.07s
823:	learn: 4.8036940	total: 14.3s	remaining: 3.05s
824:	learn: 4.8035814	total: 14.3s	remaining: 3.03s
825:	learn: 4.8035318	total: 14.3s	remaining: 3.01s
826:	learn: 4.8034977	total: 14.3s	remaining: 3s
827:	learn: 4.8033904	total: 14.3s	remaining: 2.98s
828:	learn: 4.8032963	total: 14.4s	remaining: 2.96s
829:	learn: 4.8032255	total: 14.4s	remaining: 2.94s
830:	learn: 4.8032063	total: 14.4s	remaining: 2.93s
831:	learn: 4.8031739	total: 14.4s	remaining: 2.91s
832:	learn: 4.8031274	total: 14.4s	remaining: 2.89s
833:	learn: 4.8030595	total: 14.4s	remaining: 2.87s
834:	learn: 4.80

982:	learn: 4.7952386	total: 17s	remaining: 294ms
983:	learn: 4.7951821	total: 17s	remaining: 277ms
984:	learn: 4.7951278	total: 17s	remaining: 259ms
985:	learn: 4.7950446	total: 17.1s	remaining: 242ms
986:	learn: 4.7949981	total: 17.1s	remaining: 225ms
987:	learn: 4.7949449	total: 17.1s	remaining: 208ms
988:	learn: 4.7948937	total: 17.1s	remaining: 190ms
989:	learn: 4.7948752	total: 17.1s	remaining: 173ms
990:	learn: 4.7948632	total: 17.1s	remaining: 156ms
991:	learn: 4.7948536	total: 17.2s	remaining: 138ms
992:	learn: 4.7948272	total: 17.2s	remaining: 121ms
993:	learn: 4.7948184	total: 17.2s	remaining: 104ms
994:	learn: 4.7947931	total: 17.2s	remaining: 86.5ms
995:	learn: 4.7947454	total: 17.2s	remaining: 69.2ms
996:	learn: 4.7947053	total: 17.3s	remaining: 51.9ms
997:	learn: 4.7946840	total: 17.3s	remaining: 34.6ms
998:	learn: 4.7946767	total: 17.3s	remaining: 17.3ms
999:	learn: 4.7946563	total: 17.3s	remaining: 0us


Verificando o desempenho dos modelos segundo às métricas :

In [107]:
(pd.DataFrame({'MAE' : scores_MAE , 'MSE' : scores_MSE , 'R²' : scores_R2} , index = nome_modelos)).sort_values(by = 'MAE')

,MAE,MSE,R²
LGBM,3.277265,52.090186,0.535116
Gradient Boosting,3.287891,54.111251,0.522423
Random Forest,3.309458,55.136831,0.520149
CatBoost,3.317141,58.243490,0.497389
XGB,3.336575,60.790793,0.482784
Decision Tree,3.337105,60.146558,0.486510
KNN,3.338724,57.360443,0.490462
Linear Regression,3.750121,51.306298,0.485538
ADA Boost,3.941128,58.935967,0.465221


O modelo que possui as melhores métricas é o LGBM ! Era de se esperar, pois encontramos o conjunto ótimo de features para esse modelo !

Mas veja que vários outros modelos desempenham próximo do LGBM !

Estamos conseguindo um MAE de ~ 3.28 em dados não vistos durante o treinamento do modelo ! Isso é ótimo e indica que o erro que devemos esperar ao tentarmos prever a quantidade de leads de um anúncio é de aproximadamente 3, ou seja, em média erramos para mais ou para menos em 3 anúncios.

## 4.3 Gerando Previsões em um Conjunto de Teste

Vamos criar um conjunto de treino e um de teste para podermos comparar nossas previsões com os valores reais.

In [112]:
# Criando um conjunto de treino e um de teste na mesma proporção da cross-validation a cima

X_train, X_test , y_train, y_test = train_test_split(X, y, test_size = 0.2 , random_state = 123)

In [109]:
# Instanciando, treinando e fazendo previsões

model = LGBMRegressor()

model.fit(X_train, y_train)

previsões = model.predict(X_test)

In [110]:
mean_absolute_error(y_test , previsões)

3.140539509291439

Veja que o erro absoluto médio está em linha com o da validação !

In [111]:
# Comparando valores reais com as previsões

print(pd.DataFrame({'N° verdadeiro de leads' : y_test ,  'Previsões' : previsões}).to_string())

      N° verdadeiro de leads   Previsões
0                          1   10.271232
1                          1    4.484124
2                          0    1.972642
3                          4    3.394380
4                          8    8.318844
5                          1    1.972642
6                          1    1.377064
7                          1    0.861132
8                         32   11.501408
9                          4    1.752205
10                         0    1.377064
11                         0   19.463648
12                         8    2.371246
13                         0    3.394380
14                         1    1.752205
15                         1    1.377064
16                         0    0.861132
17                         1   10.271232
18                         1    1.065462
19                         1    1.196667
20                         0    2.371246
21                         1    1.972642
22                         4    1.752205
23              

Nosso modelo está bom !

## 4.4 Otimização de Hiperparâmetros com GridSearchCV

In [113]:
# Criando a grade

grid_params = {'n_estimators' : [100, 200, 300, 400, 500, 750] , 'learning_rate' : [0.01 , 0.1 , 0.2 , 0.3] ,
               'max_depth' : [2 , 3, 4, 5]}

In [114]:
# Instanciando

gs = GridSearchCV(LGBMRegressor() , grid_params , cv = 5 , scoring = 'neg_mean_absolute_error')

In [115]:
# Treinando... essa célula demora um pouco pra rodar

gs.fit(X, y)

GridSearchCV(cv=5, estimator=LGBMRegressor(),
             param_grid={'learning_rate': [0.01, 0.1, 0.2, 0.3],
                         'max_depth': [2, 3, 4, 5],
                         'n_estimators': [100, 200, 300, 400, 500, 750]},
             scoring='neg_mean_absolute_error')

In [116]:
# Descobrindo o melhor score obtido

- gs.best_score_

3.2688013296505725

Conseguimos melhorar um pouquinho nosso MAE que foi de 3.2772 para 3.2688 .

In [117]:
# Descobrindo os parâmetros que conduzem ao melhor score

gs.best_params_

{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 300}

## 4.5 Conclusão

Concluímos que o melhor modelo é um **LGBMRegressor** treinado apenas na feature **"cliques_telefone*"** com os seguintes hiperparâmetros :

- n_estimators = 300

- learning_rate = 0.1

- max_depth = 2

Ao prevermos com esse modelo em dados não vistos durante o treinamento devemos esperar um erro médio no número de Leads de um anúncio de ~ 3,2688 leads.

Se quisermos prever um número inteiro de leads basta utilizarmos a função round() para aproximarmos nossas previsões para o inteiro mais próximo.

In [118]:
model = LGBMRegressor(learning_rate = 0.1, max_depth = 2, n_estimators = 300)
model.fit(X, y)
pred = model.predict(X)

In [119]:
# MAE no conjunto de treino

mean_absolute_error(y , pred)

3.0594373706567173

In [120]:
# MAE no conjunto de treino ao gerarmos números inteiros como previsões

previsões_inteiras = []

for previsão in pred :
    previsões_inteiras.append(round(previsão))
    

mean_absolute_error(y , previsões_inteiras)

3.004705737300674

Gerando previsões inteiras nosso score parece melhorar ainda mais !

# <center>5. Parte 2 - Modelo para determinar se um Anúncio receberá um Lead<center>

## 5.1 - Considerações Iniciais

In [121]:
# Observando o percentual de modelos que recebem pelo menos 1 lead

(sum(df['leads'] > 0)/df['leads'].count())*100

78.15017261219793

Criar um modelo com acurácia elevada não é muito difícil: Basta pensar em um modelo que prevê que todo anúncio receberá pelo menos um Lead. Como visto a cima esse modelo terá 78,15 % de acurácia, porém não conseguirá identificar corretamente nenhum dos anúncios que não recebem nenhum lead.

Sendo assim vou propor um modelo com foco em identificar corretamente o que chamaremos de **anúncios fracos, ou seja, anúncios que não recebem nenhum lead**. Assim focaremos em otimizar a métrica **Recall que mede a habilidade de um classificador em encontrar todas as amostras positivas**.

A maioria dos modelos treinados no dataset inteiro acabará "entendendo" que é melhor prever que um anúncio sempre recebe leads devido ao dataset estar fortemente desbalanceado quanto ao fato de um anúncio receber ou não um lead. Qualquer modelo terá bastante dificuldade em classificar corretamente os anúncios que não recebem leads. 

Para combater essa dificuldade precisaremos balancear as classes e para isso utilizaremos **Oversampling** que consiste em tomarmos amostras aleatórias da classe minoritária e unirmos aos dados disponíveis até que tenhamos o mesmo número de representantes em cada classe.

In [122]:
# Criando a variável Anúncio_Fraco que recebe 1 se o anúncio não recebe nenhum lead (Anúncio Fraco) e 0 caso contrário

target = []

for n_leads in df['leads'] :
    if n_leads > 0 :
        target.append(0)
    else :
        target.append(1)

df['Anúncio_Fraco'] = target

In [123]:
# Percentual de anúncios fracos

(df['Anúncio_Fraco'].mean())*100

21.84982738780207

## 5.2 Oversampling

Primeiramente vamos criar nossos conjuntos de treino e teste :

In [124]:
X = np.array(df[features])

y = np.array(df['Anúncio_Fraco'])

X_train , X_test , y_train , y_test = train_test_split(X, y , test_size = 0.2 , random_state = 90)

Vamos realizar Oversampling no conjunto de treino para balancearmos as classes, mas vamos manter o conjunto de teste desbalanceado, pois precisamos medir a performance dos nossos modelos em dados desbalanceados.

In [125]:
# Instanciando o método

ros = RandomOverSampler(sampling_strategy = 'minority')

In [126]:
X_res , y_res = ros.fit_resample(X_train , y_train)

In [127]:
# Observando quantas amostras da classe majoritária temos em y_train

sum(y_train == 0)

30403

Portanto após o oversampling devemos ter 2x30403 = 60806 amostras.

In [128]:
# Conferindo

X_res.shape[0] , y_res.shape[0]

(60806, 60806)

Tudo correto !

In [129]:
# Conferindo também se temos o mesmo número de representantes em cada classe

sum(y_res == 0) == sum(y_res == 1)

True

## 5.3 Medindo a Performance pós Oversampling

In [130]:
# Modelos

modelos = [LGBMClassifier() , KNeighborsClassifier() , DecisionTreeClassifier(),
          RandomForestClassifier() , AdaBoostClassifier(), GradientBoostingClassifier(), 
          XGBClassifier(use_label_encoder = False),
          CatBoostClassifier()]

In [131]:
# Nome dos modelos

nome_modelos = ['LGBM' , 'KNN' , 'Decision Tree' , 'Random Forest' , 'ADA Boost',
               'Gradient Boosting' , 'XGB' , 'CatBoost']

Calculando Accuracy , Precision , Recall e F1 para vários modelos :

In [132]:
# Essa célula demora um pouco pra rodar... São treinados 8 modelos em um conjunto com mais dados que antes

acc = []
precision = []
recall = []
f1 = []

for model in modelos :
    model.fit(X_res  , y_res)
    pred = model.predict(X_test)
    acc.append(accuracy_score(y_test , pred))
    precision.append(precision_score(y_test , pred))
    recall.append(recall_score(y_test , pred))
    f1.append(f1_score(y_test , pred))

[10:50:17] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.059522
0:	learn: 0.6897294	total: 78.8ms	remaining: 1m 18s
1:	learn: 0.6868005	total: 149ms	remaining: 1m 14s
2:	learn: 0.6845945	total: 239ms	remaining: 1m 19s
3:	learn: 0.6819886	total: 338ms	remaining: 1m 24s
4:	learn: 0.6796669	total: 402ms	remaining: 1m 20s
5:	learn: 0.6774867	total: 481ms	remaining: 1m 19s
6:	learn: 0.6755004	total: 567ms	remaining: 1m 20s
7:	learn: 0.6742274	total: 648ms	remaining: 1m 20s
8:	learn: 0.6728501	total: 736ms	remaining: 1m 21s
9:	learn: 0.6715769	total: 825ms	remaining: 1m 21s
10:	learn: 0.6704719	total: 937ms	remaining: 1m 24s
11:	learn: 0.6694120	total: 1.06s	remaining: 1m 27s
12:	learn: 0.6683997	total: 1.14s	remaining: 1m 26s
13:	learn: 0.6675925	total: 1.23s	remaining: 1m 26s

155:	learn: 0.6336921	total: 12.3s	remaining: 1m 6s
156:	learn: 0.6335280	total: 12.4s	remaining: 1m 6s
157:	learn: 0.6333668	total: 12.4s	remaining: 1m 6s
158:	learn: 0.6332372	total: 12.5s	remaining: 1m 6s
159:	learn: 0.6330528	total: 12.6s	remaining: 1m 6s
160:	learn: 0.6328133	total: 12.7s	remaining: 1m 5s
161:	learn: 0.6327335	total: 12.7s	remaining: 1m 5s
162:	learn: 0.6325465	total: 12.8s	remaining: 1m 5s
163:	learn: 0.6322914	total: 12.9s	remaining: 1m 5s
164:	learn: 0.6321780	total: 13s	remaining: 1m 5s
165:	learn: 0.6319875	total: 13s	remaining: 1m 5s
166:	learn: 0.6319184	total: 13.1s	remaining: 1m 5s
167:	learn: 0.6318230	total: 13.2s	remaining: 1m 5s
168:	learn: 0.6316005	total: 13.2s	remaining: 1m 5s
169:	learn: 0.6314496	total: 13.3s	remaining: 1m 4s
170:	learn: 0.6312555	total: 13.4s	remaining: 1m 4s
171:	learn: 0.6310710	total: 13.4s	remaining: 1m 4s
172:	learn: 0.6309045	total: 13.5s	remaining: 1m 4s
173:	learn: 0.6306725	total: 13.6s	remaining: 1m 4s
174:	learn: 0.63

315:	learn: 0.6046563	total: 24.2s	remaining: 52.4s
316:	learn: 0.6044958	total: 24.3s	remaining: 52.3s
317:	learn: 0.6043838	total: 24.4s	remaining: 52.2s
318:	learn: 0.6041799	total: 24.4s	remaining: 52.2s
319:	learn: 0.6039770	total: 24.5s	remaining: 52.1s
320:	learn: 0.6038398	total: 24.6s	remaining: 52s
321:	learn: 0.6036517	total: 24.7s	remaining: 51.9s
322:	learn: 0.6033939	total: 24.7s	remaining: 51.9s
323:	learn: 0.6031977	total: 24.8s	remaining: 51.8s
324:	learn: 0.6029306	total: 24.9s	remaining: 51.7s
325:	learn: 0.6027295	total: 25s	remaining: 51.6s
326:	learn: 0.6025222	total: 25s	remaining: 51.5s
327:	learn: 0.6022869	total: 25.1s	remaining: 51.4s
328:	learn: 0.6021561	total: 25.2s	remaining: 51.4s
329:	learn: 0.6020591	total: 25.3s	remaining: 51.3s
330:	learn: 0.6018822	total: 25.3s	remaining: 51.2s
331:	learn: 0.6016267	total: 25.4s	remaining: 51.1s
332:	learn: 0.6014520	total: 25.5s	remaining: 51.1s
333:	learn: 0.6013829	total: 25.6s	remaining: 51s
334:	learn: 0.601185

476:	learn: 0.5786454	total: 36.5s	remaining: 40s
477:	learn: 0.5784903	total: 36.6s	remaining: 39.9s
478:	learn: 0.5783638	total: 36.6s	remaining: 39.8s
479:	learn: 0.5782262	total: 36.7s	remaining: 39.8s
480:	learn: 0.5780067	total: 36.8s	remaining: 39.7s
481:	learn: 0.5778530	total: 36.9s	remaining: 39.6s
482:	learn: 0.5777240	total: 36.9s	remaining: 39.5s
483:	learn: 0.5776220	total: 37s	remaining: 39.5s
484:	learn: 0.5774820	total: 37.1s	remaining: 39.4s
485:	learn: 0.5773644	total: 37.1s	remaining: 39.3s
486:	learn: 0.5771893	total: 37.2s	remaining: 39.2s
487:	learn: 0.5769847	total: 37.3s	remaining: 39.1s
488:	learn: 0.5768041	total: 37.4s	remaining: 39.1s
489:	learn: 0.5766631	total: 37.5s	remaining: 39s
490:	learn: 0.5764592	total: 37.5s	remaining: 38.9s
491:	learn: 0.5763496	total: 37.6s	remaining: 38.8s
492:	learn: 0.5762087	total: 37.7s	remaining: 38.8s
493:	learn: 0.5760302	total: 37.7s	remaining: 38.7s
494:	learn: 0.5758785	total: 37.8s	remaining: 38.6s
495:	learn: 0.5757

637:	learn: 0.5556529	total: 48.5s	remaining: 27.5s
638:	learn: 0.5555363	total: 48.6s	remaining: 27.4s
639:	learn: 0.5554543	total: 48.6s	remaining: 27.4s
640:	learn: 0.5553124	total: 48.7s	remaining: 27.3s
641:	learn: 0.5551807	total: 48.8s	remaining: 27.2s
642:	learn: 0.5550452	total: 48.9s	remaining: 27.1s
643:	learn: 0.5548539	total: 48.9s	remaining: 27s
644:	learn: 0.5547159	total: 49s	remaining: 27s
645:	learn: 0.5546592	total: 49.1s	remaining: 26.9s
646:	learn: 0.5545709	total: 49.1s	remaining: 26.8s
647:	learn: 0.5544249	total: 49.2s	remaining: 26.7s
648:	learn: 0.5543352	total: 49.3s	remaining: 26.7s
649:	learn: 0.5542415	total: 49.4s	remaining: 26.6s
650:	learn: 0.5541152	total: 49.5s	remaining: 26.5s
651:	learn: 0.5539387	total: 49.5s	remaining: 26.4s
652:	learn: 0.5537902	total: 49.6s	remaining: 26.4s
653:	learn: 0.5536931	total: 49.7s	remaining: 26.3s
654:	learn: 0.5535707	total: 49.7s	remaining: 26.2s
655:	learn: 0.5534677	total: 49.8s	remaining: 26.1s
656:	learn: 0.5533

798:	learn: 0.5350419	total: 1m	remaining: 15.3s
799:	learn: 0.5348738	total: 1m	remaining: 15.2s
800:	learn: 0.5347309	total: 1m	remaining: 15.1s
801:	learn: 0.5346052	total: 1m	remaining: 15s
802:	learn: 0.5345168	total: 1m	remaining: 14.9s
803:	learn: 0.5344269	total: 1m 1s	remaining: 14.9s
804:	learn: 0.5343469	total: 1m 1s	remaining: 14.8s
805:	learn: 0.5341931	total: 1m 1s	remaining: 14.7s
806:	learn: 0.5340469	total: 1m 1s	remaining: 14.6s
807:	learn: 0.5338920	total: 1m 1s	remaining: 14.6s
808:	learn: 0.5337604	total: 1m 1s	remaining: 14.5s
809:	learn: 0.5336486	total: 1m 1s	remaining: 14.4s
810:	learn: 0.5335417	total: 1m 1s	remaining: 14.3s
811:	learn: 0.5334477	total: 1m 1s	remaining: 14.3s
812:	learn: 0.5333414	total: 1m 1s	remaining: 14.2s
813:	learn: 0.5332058	total: 1m 1s	remaining: 14.1s
814:	learn: 0.5331063	total: 1m 1s	remaining: 14s
815:	learn: 0.5330112	total: 1m 1s	remaining: 14s
816:	learn: 0.5329028	total: 1m 1s	remaining: 13.9s
817:	learn: 0.5328165	total: 1m 2

956:	learn: 0.5164486	total: 1m 12s	remaining: 3.27s
957:	learn: 0.5163141	total: 1m 12s	remaining: 3.19s
958:	learn: 0.5162106	total: 1m 12s	remaining: 3.12s
959:	learn: 0.5161100	total: 1m 12s	remaining: 3.04s
960:	learn: 0.5160401	total: 1m 13s	remaining: 2.96s
961:	learn: 0.5159016	total: 1m 13s	remaining: 2.89s
962:	learn: 0.5157327	total: 1m 13s	remaining: 2.81s
963:	learn: 0.5156401	total: 1m 13s	remaining: 2.73s
964:	learn: 0.5155592	total: 1m 13s	remaining: 2.66s
965:	learn: 0.5154523	total: 1m 13s	remaining: 2.58s
966:	learn: 0.5152771	total: 1m 13s	remaining: 2.51s
967:	learn: 0.5152052	total: 1m 13s	remaining: 2.43s
968:	learn: 0.5150968	total: 1m 13s	remaining: 2.35s
969:	learn: 0.5149831	total: 1m 13s	remaining: 2.28s
970:	learn: 0.5148529	total: 1m 13s	remaining: 2.2s
971:	learn: 0.5147006	total: 1m 13s	remaining: 2.13s
972:	learn: 0.5145685	total: 1m 13s	remaining: 2.05s
973:	learn: 0.5144866	total: 1m 13s	remaining: 1.97s
974:	learn: 0.5143491	total: 1m 14s	remaining: 

In [133]:
# Exibindo os resultados obtidos

pd.DataFrame({'Recall' : recall , 'Acurácia' : acc , 'Precision' : precision , 'F1' : f1}, index = nome_modelos).sort_values(
by = 'Recall' , ascending = False)

,Recall,Acurácia,Precision,F1
Gradient Boosting,0.749169,0.487722,0.261309,0.387469
ADA Boost,0.709739,0.508579,0.263678,0.384506
LGBM,0.638480,0.527895,0.259560,0.369079
CatBoost,0.487411,0.600534,0.267536,0.345455
KNN,0.467458,0.548752,0.231257,0.309434
XGB,0.467458,0.591185,0.256117,0.330923
Decision Tree,0.248931,0.671016,0.244289,0.246588
Random Forest,0.029454,0.775609,0.305419,0.053726


**Agora é tudo uma questão de objetivo do nosso modelo**. Por exemplo, **se o objetivo for ter o maior número de acertos quanto a um anúncio receber ou não Leads o melhor modelo seria o de maior acurácia que nesse caso é o Random Forest**. Porém, nesse caso não estaríamos nos preocupando em identificar corretamente os anúncios que não recebem leads já que o recall do RF é baixíssimo. Por outro lado, **o Gradient Boosting classifica corretamente 74,92 % dos anúncios fracos , mas classifica somente aproximadamente 48,77 % do total de anúncios corretamente**.

**Como acredito ser importantíssimo identificar quem são os anúncios que não recebem leads acabo optando pelo Gradient Boosting ainda que esse tenha apenas 50 % de acurácia**. Não existem muitos problemas quanto ao modelo classificar errôneamente anúncios fortes, porém deixando de detectar os anúncios fracos a empresa não poderia explicar para seus clientes como proceder para não montar um anúncio fraco.

## 5.4 Feature Selection 

In [134]:
# Treinando um Gradient Boosting

model = GradientBoostingClassifier()
model.fit(X_res  , y_res)
pred = model.predict(X_test)

In [135]:
# Obtendo a importância das Features

imps = model.feature_importances_

In [136]:
pd.DataFrame({'Feature' : features , '% de Importância' : imps}).sort_values(by = '% de Importância' , ascending = False)

,Feature,% de Importância
5,cliques_telefone*,0.613711
10,cep_2dig,0.088418
0,cod_anuncio,0.043763
1,cod_cliente,0.038147
13,km_veiculo,0.029374
...,...,...
47,gas natural,0.000000
50,gasolina e eletrico,0.000000
51,gasolina e gas natural,0.000000
53,"gasolina, alcool, gas natural e benzina",0.000000


Novamente a feature mais importante para o modelo é "cliques_telefone*" . Será que se treinarmos um GradientBoosting nas duas features mais importantes (juntas reúnem pouco mais 70% da importância total) teremos um melhor recall ? É o que vamos verificar abaixo :

In [137]:
# Treinando e obtendo as métricas

model = GradientBoostingClassifier()
model.fit(X_res[: , [5, 10]] , y_res)
pred = model.predict(X_test[: , [5,10]])
acurácia = accuracy_score(y_test, pred)
precisão = precision_score(y_test, pred)
recall = recall_score(y_test, pred)
f1 = f1_score(y_test, pred)

In [138]:
pd.DataFrame({'Recall' :[recall], 'Acurácia' : [acurácia] , 'Precisão' : [precisão] , 'F1' : [f1]})

,Recall,Acurácia,Precisão,F1
0,0.811401,0.45392,0.257772,0.39125


A acurácia diminui um pouco, mas conseguimos obter 81,14 % de recall !

Proponho então o uso do seguinte modelo :

**Gradient Boosting treinado nas features "cliques_telefone*" e "cep_2dig"**

Observação : O modelo entrega ~ 81 % de recall, ou seja, classifica anúncios fracos corretamente em 81 % dos casos apesar de ter uma acurácia de 45,39%. Cumpre bem o papel de achar quem são os anúncios fracos.

## 5.5 O que poderia nos levar a conclusões melhores ?

O grande problema do dataset em questão é que ele é desbalanceado, ou seja, a quantidade de anúncios em que houve pelo menos 1 lead é muito maior do que a quantidade de anúncios em que não houve nenhum lead. A proporção é de aproximadamente 4 : 1 . Para termos modelos melhores precisaríamos de mais amostras referentes a anúncios que não tiveram nenhum lead, pois assim conseguiriamos balancear mais os dados e o modelo conseguiria aprender a distinguir as duas classes ao invés de prever sempre 0. A distribuição dos anúncios dificulta o treinamento dos modelos, pois eles tendem a "entender" que é melhor prever que sempre há pelo menos um lead afinal um modelo que prevê isso obtém alta acurácia (~ 80%).

A técnica Oversampling permite que consigamos contornar um pouco isso e classificar corretamente um número maior de anúncios fracos, mas ela tem o preço de diminuir a acurácia dos nossos modelos. Além disso, durante o Oversampling inserimos amostras da classe minoritária, porém essas amostras não são distintas das já existentes e assim temos uma série de amostras repetidas a fim de igualar a quantidade de elementos da mesma classe. Isso pode levar ao overfitting uma vez que o modelo vê o mesmo exemplo mais de uma vez e, por isso, pode simplesmente memorizar o conjunto de treino e não conseguir generalizar para dados não vistos durante o treinamento.